Data Wrangling Notebook for Bernor Equid dataset
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [4]:
import pandas as pd
import numpy as np
import uuid

Read original data 

In [5]:
ray_data = pd.read_csv("../Original_Data/ToFuTRESVER_14_7_27_2020_REV_22.csv")
locality_data = pd.read_csv("../Original_Data/LOCAL_7_22_2020FuTRESVer12.csv")

Standardize 'SEX' column 

In [6]:
female = ray_data['SEX']=="female"
male = ray_data['SEX'] == "male"
ray_data['SEX'][(female == False)&(male==False)]="not collected"

<ipython-input-6-7e7ba66aaf77>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data['SEX'][(female == False)&(male==False)]="not collected"


Standardize "SIDE" column

In [7]:
# Clean up Side column 
right = ray_data['SIDE']=="right"
left = ray_data['SIDE'] == "left"
ray_data['SIDE'][(female == False)&(male==False)]="not collected"

<ipython-input-7-0a21778ecf0f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data['SIDE'][(female == False)&(male==False)]="not collected"


Combine genus and species column to create verbatimScientificName column (not currently accepted in GEOME)

In [9]:
# Leave out of final dataframe for now. Use to standardize scientificName
ray_data = ray_data.assign(verbatimScientificName = ray_data['GENUS'] + " " + ray_data['SPECIES'])

Create standardized species column

In [11]:
ray_data.verbatimScientificName.unique()

#ray_data["scientificName"] = ray_data["scientificName"].fillna("unknown")

array(['Equus quagga', 'Equus grevyi', 'Equus quagga boehemi',
       'Cormohipparion sp.', 'Equus caballus',
       'Cormohipparion cf. occidentale', 'Cormohipparion matthewi',
       'Cormohipparion occidentale', 'Calippus sp.', 'Calippus sp..',
       'Cormohipparion sp. LG', 'Pseudhipparion sp.',
       'Cormohipparion sp.2', 'Cormohipparion fricki', 'Merychippus sp.',
       'Merychippus  insignis', 'Cormohipparion goorisi',
       'Cormohipparion sphenodus', 'Cormohipparion johnsoni',
       'Cormohipparion sp.1', 'Cormohipparion skinneri',
       'Equus conversidens', 'Equus occidentalis', 'Equus simplicidens',
       'Equus asinus', '"Hipparion"  sp.', 'Cremohipparion licenti',
       'Plesiohipparion shanxiense', 'Proboscidipparion pater',
       'Baryhipparion cf.tchicoisum', 'Baryhipparion insperatum ',
       'Baryhipparion insperatumT?', 'Plesiohipparion houfenense',
       'Proboscidipparion sinense', 'Equus eisenmannae',
       'Sivalhippus platyodus L', 'Hipparion coelo

In [91]:
# Create verbatimEventData column 
ray_data = ray_data.assign(verbatimEventDate = ray_data['DATE COLLECTED'])

# Create yearCollected column 
ray_data = ray_data.assign(yearCollected = "")

date_filter=ray_data["verbatimEventDate"].str.contains("Lower|Upper|Uppermost|Loc")
ray_data["yearCollected"]=ray_data["verbatimEventDate"][date_filter==False]

ray_data['yearCollected'] = ray_data.yearCollected.str[-4:]
ray_data['yearCollected'] = ray_data['yearCollected'].fillna("unknown")

In [92]:
# Create individualID column
ray_data = ray_data.assign(individualID = ray_data['SPEC_ID'])

In [93]:
# Create verbatimAgeValue columns 
ray_data = ray_data.assign(verbatimAgeValue = ray_data['AGE'])
ray_data = ray_data.assign(minimumChronometricAgeReferenceSystem ="mya")
ray_data = ray_data.assign(maximumChronometricAgeReferenceSystem ="mya")


# Create columns for minimumChronometricAge and maximumChronometricAge
ray_age_ranges = ray_data["verbatimAgeValue"].str.contains("-")
ray_ranges = ray_data["verbatimAgeValue"][ray_age_ranges==True]
ray_data = ray_data.assign(minimumChronometricAge = "")
ray_data = ray_data.assign(maximumChronometricAge = "")

for ind in ray_ranges.index:
    x = ray_ranges[ind]
    y = str(x)
    z = str(y).split("-")
    
    if z[0] > z[1]:
        ray_data['maximumChronometricAge'][ind] = z[0]
        ray_data['minimumChronometricAge'][ind] = z[1]
    else:
        ray_data['maximumChronometricAge'][ind] = z[1]
        ray_data['minimumChronometricAge'][ind] = z[0]
    
        
        

<ipython-input-93-a5f4f72c777c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data['maximumChronometricAge'][ind] = z[1]
<ipython-input-93-a5f4f72c777c>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data['minimumChronometricAge'][ind] = z[0]
<ipython-input-93-a5f4f72c777c>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data['maximumChronometricAge'][ind] = z[0]
<ipython-input-93-a5f4f72c777c>:20: SettingWithCopyW

In [94]:
#Fix Country and Locality by matching to Locality sheet

ray_data = ray_data.assign(country = "")
ray_data = ray_data.assign(verbatimLocality = "")
ray_data = ray_data.assign(ray_temp_locality = "")

ray_data["COUNTRY"] = ray_data["COUNTRY"].apply(str)

ray_data['ray_temp_locality'] = ray_data['COUNTRY']+" "+ ray_data['LOCALITY']
locality_data['local_temp_locality'] = locality_data['COUNTRY No'] + " " + locality_data["LOCALITY No."]

for i in ray_data.index:
    for j in locality_data.index:
        if ray_data['ray_temp_locality'][i] == locality_data['local_temp_locality'][j]:
            ray_data["country"][i] = locality_data["COUNTRYName"][j]
            ray_data["verbatimLocality"][i] = locality_data["LOCALITYName"][j]
        else:
            ray_data["country"][i] = "Unknown"

            
ray_data['country']=ray_data['country'].replace({'Tibetan Plateau, Nepal':'Nepal'}) 

ray_data=ray_data.assign(locality= ray_data["verbatimLocality"]).fillna("Unknown")


<ipython-input-94-7ea5ec9e5bf4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data["country"][i] = locality_data["COUNTRYName"][j]
<ipython-input-94-7ea5ec9e5bf4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data["verbatimLocality"][i] = locality_data["LOCALITYName"][j]
<ipython-input-94-7ea5ec9e5bf4>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ray_data["country"][i] = "Unknown"


In [95]:
# Rearrange columns so that template columns are first, followed by measurement values

# Create column list
cols = ray_data.columns.tolist()

# Specify desired columns
cols = ['individualID',
        'SEX',
        'SIDE',
        'scientificName',
        'verbatimEventDate',
        'yearCollected',
        'verbatimAgeValue',
        'minimumChronometricAge',
        'maximumChronometricAge',
        'minimumChronometricAgeReferenceSystem',
        'maximumChronometricAgeReferenceSystem',
        'verbatimLocality',
        'locality',
        'country',
        'BONE',
        'M1',
        'M2',
        'M3',
        'M4',
        'M5',
        'M6',
        'M7',
        'M8',
        'M9',
        'M10',
        'M11',
        'M12',
        'M13',
        'M14',
        'M15',
        'M16',
        ' M17',
        'M18',
        'M19',
        'M20',
        'M21',
        'M22',
        'M23',
        'M24',
        'M25',
        'M26',
        'M27',
        'M28',
        'M29',
        'M30',
        'M31',
        'M32',
        'M33',
        'M34',
        'M35',
        'M36',
        'M37',
        'M38']

# Subset dataframe
ray_data = ray_data[cols]

In [96]:
# Matching template and column terms

# Renaming columns 
ray_data = ray_data.rename(columns = {'SEX':'sex',
                                      'SIDE':'side'})

In [97]:
# Create measurementUnit column
ray_data = ray_data.assign(measurementUnit="in")

ray_data["locality"]=ray_data["locality"].fillna("Unknown")

In [98]:
# Create basisOfRecord column
ray_data=ray_data.assign(basisOfRecord="")

recent_filter = ray_data['verbatimAgeValue']=="recent"
not_recent_filter = ray_data['verbatimAgeValue']!="recent"

ray_data['basisOfRecord'][recent_filter] = "PreservedSpecimen"
ray_data['basisOfRecord'][not_recent_filter] = "FossilSpecimen"


In [99]:
# Fill in blanks for required columns 
ray_data=ray_data.assign(samplingProtocol="Unknown")
ray_data=ray_data.assign(measurementMethod="Unknown")

In [100]:
# Create materialSampleID which is a UUID for each measurement
# Create eventID and populate it with materialSampleID

ray_data=ray_data.assign(materialSampleID = '')
ray_data['materialSampleID'] = [uuid.uuid4() for _ in range(len(ray_data.index))]

for ind in ray_data.index:
    x=ray_data['materialSampleID'][ind]
    y=str(x)
    z=y.replace("-", '_')
    
    ray_data['materialSampleID'][ind] = z

ray_data=ray_data.assign(eventID = ray_data["materialSampleID"])

In [101]:
#  Create long version so that each trait has its own row

# Creating long version, first specifiying keep variables, then naming variable and value
longVers=pd.melt(ray_data, 
                id_vars=['individualID',
                         'sex',
                         'side',
                         'scientificName',
                         'verbatimEventDate',
                         'yearCollected',
                         'verbatimAgeValue',
                         'minimumChronometricAge',
                         'maximumChronometricAge',
                         'minimumChronometricAgeReferenceSystem',
                         'maximumChronometricAgeReferenceSystem',
                         'verbatimLocality',
                         'locality',
                         'country',
                         'measurementUnit',
                         'basisOfRecord',
                         'samplingProtocol',
                         'measurementMethod',
                         'materialSampleID',
                         'eventID',
                         'BONE'], 
                var_name = 'measurementType', 
                value_name = 'measurementValue')
longVers

,individualID,sex,side,scientificName,verbatimEventDate,yearCollected,verbatimAgeValue,minimumChronometricAge,maximumChronometricAge,minimumChronometricAgeReferenceSystem,...,country,measurementUnit,basisOfRecord,samplingProtocol,measurementMethod,materialSampleID,eventID,BONE,measurementType,measurementValue
0,SMNS1508,not collected,not collected,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,e8268688_7317_4c38_a561_414e03d5fd8d,e8268688_7317_4c38_a561_414e03d5fd8d,a1ph3,M1,77.2
1,SMNS7335,male,unknown,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,8cee7142_036d_4b52_80cc_5be52c76697a,8cee7142_036d_4b52_80cc_5be52c76697a,a2ph3,M1,39.4
2,SMNS1508,not collected,not collected,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,59ed168b_2be5_44ed_b607_7f34b2b362d0,59ed168b_2be5_44ed_b607_7f34b2b362d0,astragalus,M1,55.7
3,SMNS1508,not collected,not collected,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,6608088a_645d_4319_bbef_6b176ffcb720,6608088a_645d_4319_bbef_6b176ffcb720,calcaneum,M1,101.9
4,SMNS7335,male,unknown,Equus quagga,Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,953f4264_be2e_4d77_8cc0_5c25781d4dda,953f4264_be2e_4d77_8cc0_5c25781d4dda,femur,M1,357.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807267,USNM541427,female,left,Equus hemionus,10-Jul-1981,1981,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,fed0c549_4bca_46a1_9123_3378e8cf4206,fed0c549_4bca_46a1_9123_3378e8cf4206,txP4,M38,Unknown
807268,MSNAF2842,not collected,not collected,"""Hipparion"" sp.",Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,7ea0896f_7dfb_42f8_88eb_4b9fd31d9062,7ea0896f_7dfb_42f8_88eb_4b9fd31d9062,txP3,M38,Unknown
807269,MSNAF2842,not collected,not collected,"""Hipparion"" sp.",Unknown,unknown,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,71443f16_90d1_49b0_af1f_c5eec651a72b,71443f16_90d1_49b0_af1f_c5eec651a72b,txP4,M38,Unknown
807270,USNM241009,male,left,Equus grevyi,9-Oct-1925,1925,0.0,,,mya,...,Unknown,in,FossilSpecimen,Unknown,Unknown,dba7c312_5b8a_4762_bac3_27f7b64cd4e3,dba7c312_5b8a_4762_bac3_27f7b64cd4e3,tml1,M38,Unknown


In [102]:
# Create new measurementType column by combining BONE and measurementType column
longVers['measurementType'] = longVers['BONE'] + longVers['measurementType']
longVers['measurementType']

0              a1ph3M1
1              a2ph3M1
2         astragalusM1
3          calcaneumM1
4              femurM1
              ...     
807267         txP4M38
807268         txP3M38
807269         txP4M38
807270         tml1M38
807271      UnknownM38
Name: measurementType, Length: 807272, dtype: object

In [103]:
# Remove BONE columnn
del longVers['BONE']

In [104]:
# Filter out the GEOME measurements
correct_element_filter = longVers["measurementType"].str.match("femurM1|femurM2|humerusM1|humerusM2|astragalusM3|astraglausM2|astragalusM7|astragalusM1|astragalusM4|calcaneumM1|calcaneumM6")
incorrect_filter = longVers["measurementType"].str.contains("M10|M11|M12|M13|M14|M15|M16|M17|M18|M19|M20|M21|M22|M23|M24|M25|M26|M27|M28|M29|M30|M31|M32|M33|M34|M35|M36|M37|M38")
longVers=longVers[correct_element_filter==True][incorrect_filter==False]

<ipython-input-104-1c0a172043b4>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  longVers=longVers[correct_element_filter==True][incorrect_filter==False]


In [105]:
# Renaming measurementType values

longVers['measurementType']=longVers['measurementType'].replace({'femurM1': 'femur length to greater trochanter', 
                                                                 'femurM2': 'femur length to head of femur', 
                                                                 'humerusM1': 'humerus length to ventral tubercle', 
                                                                 'humerusM2': 'humerus length to caput of humerus',
                                                                 'astragalusM3':'breadth of the trochlea of talus',
                                                                 'astragalusM4':'talus breadth',
                                                                 'astraglausM2':'length of medial trochlea of talus',
                                                                 'astragalusM7':'depth of the medial side of talus',
                                                                 'astragalusM1':'astragalus length',
                                                                 'calcaneumM1' : 'calcaneus length',
                                                                 'calcaneumM6' : 'calcaneus breadth'
                                                                 #'astragalusM1':'length of the medial side of talus'
                                                                 #'astragalusM5':'distal articular breadth of talus',
                                                                 #'astragalusM6':'distal articular depth of talus'
                                                                 })


In [106]:
# Create diagnosticID which is a unique number for each measurement
longVers=longVers.assign(diagnosticID = '')
longVers['diagnosticID'] = np.arange(len(longVers))

In [107]:
# Fix up measurementValue entries
#measurement_value_filter=longVers["measurementValue"].str.contains('frag|E|lat|NOT')
zero_values=longVers["measurementValue"]=="0"
blanks = longVers["measurementValue"] ==""
unknown = longVers["measurementValue"] == "Unknown"
locality_blanks = longVers["locality"]==""

longVers["measurementValue"]=longVers["measurementValue"][zero_values==False][blanks==False][unknown == False]
longVers["locality"][locality_blanks]="unknown"



<ipython-input-107-b6ff83af824b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longVers["locality"][locality_blanks]="unknown"


In [108]:
# If measurement value equals N/a, delete entire row
longVers = longVers.dropna(subset=['measurementValue'])

# Drop first row of data, it contains no measurementValue but is still retained
longVers = longVers.drop(longVers.index[0])

In [109]:
# Writing long data csv file
longVers.to_csv('../Mapped_Data/FuTRES_Equid_Bernor_Global_Cenozoic.csv', index = False);

In [110]:
#Subset needed:
#talus breadth
#astragalus length
#calcaneus length
#calcaneus breadth
#humerus length to ventral tubercle
#humerus length to caput of humerus


ray_subset = longVers[(longVers["measurementType"] == "talus breadth") | 
                      (longVers["measurementType"] == "astragalus length") |
                      (longVers["measurementType"] == "calcaneus length") |
                      (longVers["measurementType"] == "calcaneus breadth") |
                      (longVers["measurementType"] == "humerus length to ventral tubercle") |
                      (longVers["measurementType"] == "humerus length to caput of humerus")]

In [111]:
ray_subset.to_csv('../Mapped_Data/FuTRES_Equid_Bernor_Subset.csv', index = False)

In [4]:
ray_data_orig = pd.read_csv("../Original_Data/ToFuTRESVER_14_7_27_2020_REV_22.csv")
len(ray_data_orig)

21244

In [6]:
ray_new = pd.read_csv('../Mapped_Data/FuTRES_Equid_Bernor_Subset.csv')
len(ray_new)

2066